In [ ]:
pip install selenium

In [1]:
handle = '' #인스타그램 아이디
pwd = ''  #인스타그램 비밀번호
keyword= '치즈케이크'   #검색할 키워드
num_of_pages = 1000 #몇 번 스크롤 할 건지; e.g. 200 --> 1730개 게시글 긁어짐

In [9]:
#함수 정의
from selenium import webdriver
from urllib.parse import quote
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd 
import re

#URL 가져오기
def InstagramUrlFromKeyword (browser,keyword,num_of_pagedowns):
    keyword_url_encode=quote(keyword)
    url='https://www.instagram.com/explore/tags/'+keyword+'/?hl=ko'
    browser.get(url)
    time.sleep(3)
    login = browser.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button')
    login.click()
    time.sleep(3)
    user_id = browser.find_element_by_class_name('_2hvTZ.pexuQ.zyHYP')
    pw = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
    user_id.send_keys(handle)
    pw.send_keys(pwd)
    button = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button')
    button.click()
    time.sleep(3)
    arr_href=[]
    body=browser.find_element_by_tag_name('body')
    for i in range(num_of_pagedowns):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        post=browser.find_elements_by_class_name('v1Nh3')
        for j in post:
            href_str=j.find_element_by_css_selector('a').get_attribute('href')
            arr_href.append(href_str)
    return set(arr_href)    

char = r'#[\d\w]+'
char2 = re.compile('[^ 0-9a-zA-Zㄱ-ㅣ가-힣!#?]')

#URL에서 데이터 가져오기
def IdHashTagFromInstagram(browser,url):
    browser.get(url)
    datetime = browser.find_element_by_class_name('_1o9PC.Nzb55').get_attribute('title')
    href=browser.find_elements_by_class_name('C4VMK')
    #포스트 내용(글, 태그)
    total_hash_text=[]    
    for i in range(0, len(href)):
        hash_text= href[i].find_element_by_css_selector('span').text
        total_hash_text.append(hash_text)
    tags = re.findall(char, char2.sub(' ',str(total_hash_text)))
    user_name = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[1]/div[1]/h2/a').text
    like_count = int(browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/div[2]/section[2]/div/div/button/span').text)
    location = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a').text
    return datetime, user_name, like_count, tags, location

In [3]:
#크롤링
browser= webdriver.Chrome()
arr=InstagramUrlFromKeyword(browser, keyword, num_of_pages)


In [ ]:
insta_df=pd.DataFrame(columns = {'Datetime', 'User_Name', 'Like_Count', 'Tags', 'Location'})

for url in arr:
        try:
            datetime, user_name, like_count, tags, location = IdHashTagFromInstagram(browser,url)
            insta_df = insta_df.append({'Datetime': datetime, 'User_Name':user_name, 'Like_Count':like_count,'Tags': tags, 'Location' : location}, ignore_index = True)
        except:
            continue

In [7]:
arr

{'https://www.instagram.com/p/B6dWr2zljnx/',
 'https://www.instagram.com/p/B6VVsjxFj_o/',
 'https://www.instagram.com/p/B6dCwmfHr9b/',
 'https://www.instagram.com/p/B6LiK4sH9b6/',
 'https://www.instagram.com/p/B6pr1iZHd8-/',
 'https://www.instagram.com/p/B6iK1vOhw58/',
 'https://www.instagram.com/p/B6qIMZdprLK/',
 'https://www.instagram.com/p/B6GKLZvj4aI/',
 'https://www.instagram.com/p/B6lA5jplvwN/',
 'https://www.instagram.com/p/B6K42AJFssn/',
 'https://www.instagram.com/p/B52YCJZlz75/',
 'https://www.instagram.com/p/B6aBT51nBFW/',
 'https://www.instagram.com/p/B5_iQ0-FbfQ/',
 'https://www.instagram.com/p/B6R1OHXgjPC/',
 'https://www.instagram.com/p/B6jv4_ggJgJ/',
 'https://www.instagram.com/p/B6hwkYsl-66/',
 'https://www.instagram.com/p/B6Now4RhwJQ/',
 'https://www.instagram.com/p/B56-NhRg3mm/',
 'https://www.instagram.com/p/B6iBxjLpD3d/',
 'https://www.instagram.com/p/B6N-DjdlqGY/',
 'https://www.instagram.com/p/B6kk5FtJjg0/',
 'https://www.instagram.com/p/B6Sdr8DlOPi/',
 'https://

In [10]:
IdHashTagFromInstagram(browser, 'https://www.instagram.com/p/B6EVCOdn1dB/')

('2019년 12월 15일', 'rora0220', 62, [], 'Dirty Trunk')

In [5]:
len(arr)

7761

In [6]:
insta_df

,Like_Count,Location,Datetime,Tags,User_Name


In [ ]:
#csv 저장
insta_df.to_csv(keyword + '.csv',mode='w', encoding='euc-kr')

In [13]:
#빈도분석
from collections import Counter

insta_str = []
for tags in insta_df['Tags']:
    insta_str.extend(tags)
    
c = Counter(insta_str)

c.most_common()

[('#선릉맛집', 1472),
 ('#강남맛집', 465),
 ('#선릉역맛집', 325),
 ('#대치동맛집', 256),
 ('#먹스타그램', 244),
 ('#맛스타그램', 236),
 ('#일상', 236),
 ('#퍼블릭', 197),
 ('#역삼맛집', 195),
 ('#선릉', 189),
 ('#데일리', 182),
 ('#강남', 150),
 ('#서울맛집', 144),
 ('#광안리맛집', 134),
 ('#맛집', 133),
 ('#해운대맛집', 133),
 ('#소통', 129),
 ('#맞팔', 122),
 ('#회식', 118),
 ('#삼성동맛집', 117),
 ('#f4f', 105),
 ('#모임', 102),
 ('#서면맛집', 101),
 ('#가라오케', 100),
 ('#접대', 100),
 ('#강남가라오케', 99),
 ('#강남셔츠룸', 98),
 ('#강남유흥', 98),
 ('#유흥', 98),
 ('#유앤미', 98),
 ('#런닝레빗', 98),
 ('#생일파티', 98),
 ('#밤일', 98),
 ('#강남란제리', 98),
 ('#알바', 98),
 ('#고액알바', 98),
 ('#단기알바', 98),
 ('#고소득알바', 98),
 ('#유흥알바', 98),
 ('#급전', 98),
 ('#전포동맛집', 96),
 ('#먹방', 95),
 ('#선릉카페', 93),
 ('#회식장소', 91),
 ('#민락동맛집', 87),
 ('#해운대술집', 87),
 ('#센텀맛집', 87),
 ('#문정동맛집', 87),
 ('#미스터스시', 86),
 ('#광안리술집', 86),
 ('#남천동맛집', 86),
 ('#센텀시티맛집', 86),
 ('#재송동맛집', 86),
 ('#서면술집', 86),
 ('#서면밥집', 86),
 ('#가락동맛집', 86),
 ('#거제도맛집', 86),
 ('#거제맛집', 86),
 ('#괴정동맛집', 86),
 ('#송파맛집', 86),
 ('#옥포맛집', 86),
 ('#디